In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SequentialChain, LLMChain

load_dotenv()

True

In [40]:
import neo4j

host = "localhost"
username = "neo4j"
password = "1234qwer"

driver = neo4j.GraphDatabase.driver("neo4j://100.27.33.222:7687",
  auth=neo4j.basic_auth("neo4j", "price-oxygens-scores")
)
# driver = neo4j.GraphDatabase.driver(f'bolt://localhost:7687', auth=(username, password))
session = driver.session()

def create_graph(query):
    session.run(query)

In [14]:
template1 = """You are expert english linguists 

Given sentence : $$$ {sentence} $$$

- Generate semantic triple that consist subject, predicate, and object  base on context sentence (which is delimited by $$$) 
- Output must separated by comma the format is [subject],[predicate],[object]
- Avoid using pronouns such as (you, he, she, it, we, they etc), replace with more relevant subject
- Each line is separated by '\n'
- Don't display anything in the output except the output csv format

"""

prompt1 = ChatPromptTemplate.from_template(template1)

model = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv('OPEN_API_KEY'), temperature=0)

chain1 = prompt1 | model

chain_one = LLMChain(llm=model, prompt=prompt1, output_key="triple")

overal_chain = SequentialChain(
    chains=[chain_one],
    input_variables=["sentence"],
    output_variables=["triple"]
)

In [ ]:
import pandas as pd
import csv

df_dataset = pd.read_csv('output/datasets_all_sentence.csv')
for index, row in df_dataset.iterrows():
    sentence = row['sentence']
    app = row['app']
    uid = row['id']
    feature = row['feature']
    message = overal_chain({"sentence": sentence})
    triples = message['triple'].split('\n')

    for row in triples:
        with open("data/all_triplet_lang_result.csv", "a", newline="\n") as csvfile:
            writer = csv.writer(csvfile)
            row = f"{uid},{app},{feature},{row}"
            writer.writerow(row.split(","))

In [41]:
import pandas as pd
df_dataset = pd.read_csv('output/datasets_all_sentence.csv')

In [ ]:
def df_to_create_cql(df, uid):
    cql_query = "CREATE "
    entities = []
    for index, row in df.iterrows():
        if (row['subject'] in entities) == False:
            entities.append(row['subject'].strip())
        if (row['object'] in entities) == False:
            entities.append(row['object'].strip())
    init = 'a'
    obj = {}
    for i in range(len(entities)):
        char = chr(ord(init) + i)
        obj[entities[i]] = char
        cql_query += f"({char}:CORE" + "{" + f"id: '{uid}', name: '{entities[i]}'" + "}), \n"

    idx = 1
    for _, row in df.iterrows():
        subjek = obj[row['subject'].strip()]
        objek = obj[row['object'].strip()]
        predicate = row['predicate'].strip()
        cql_query += f"({subjek})-[:RELATED_TO " + "{ type:"+ f"'{predicate}'" + "}]" + f"->({objek})"
        if(len(df) != idx):
            cql_query += ", \n"
        idx += 1
    return cql_query

In [43]:
import pandas as pd
df_result = pd.read_csv('data/all_triplet_lang_result.csv')

for index, row in df_dataset.iterrows():
    try:
        df_filter = df_result[df_result['id'] == row['id']]
        cql_query = df_to_create_cql(df_filter, row['id'])
        session.run(cql_query)
    except Exception as e:
        print(row)
        break

In [44]:
from transformers import RobertaTokenizer, RobertaModel

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaModel.from_pretrained("roberta-base")

/Users/fahmisyaifudin/miniconda3/envs/llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
def bert_embedding(text):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    pooler_output = output['pooler_output'][0]
    return pooler_output.detach().numpy()

In [ ]:
for index, row in df_dataset.iterrows():
    uid = row['id']
    records, summary, key = driver.execute_query("MATCH(n {id: $id})-[k]->(m {id: $id}) RETURN n.name, k.type, m.name", id=uid)
    for record in records:
        embedding_subject = bert_embedding(record['n.name'])
        session.run("MATCH (n {id: $id, name: $name}) WHERE n.embedding is null SET n.embedding = $embedding", id=uid, name=record['n.name'], embedding=embedding_subject)
        embedding_object = bert_embedding(record['m.name'])
        session.run("MATCH (m {id: $id, name: $name}) WHERE m.embedding is null SET m.embedding = $embedding", id=uid, name=record['m.name'], embedding=embedding_object)
        embedding_relation = bert_embedding(record['k.type'])
        session.run("MATCH(n {id: $id})-[k {type: $name}]->(m {id: $id}) WHERE k.embedding is null SET k.embedding = $embedding", id=uid, name=record['k.type'], embedding=embedding_relation)

In [ ]:
for index, row in df_dataset.iterrows():
    session.run("MATCH (n {id: $id}) SET n.app = $app, n.feature = $feature", app=row['app'], id=row['id'], feature=row['feature'])